# Create Model

In [1]:
import numpy as np
import pandas as pd

from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("image_data.csv", header=None)
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset

In [3]:
# data[0] = data[0].map(lambda x: x[-1])
# data.to_csv('image_data.csv', header=None, index=False)

In [4]:
image_x = 50
image_y = 50

#2501 because we have images of size 50x50
X = X[:, 1:(image_x*image_y+1)]
Y = Y[:, 0]

In [5]:
data.shape

(6000, 2501)

In [6]:
X_train = X[0:5000, :]
X_train = X_train/255.
X_test = X[5000:6001, :]
X_test = X_test/255.

In [7]:
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:5000, :]
Y_train = Y_train.T
Y_test = Y[5000: 6001, :]
Y_test = Y_test.T

In [8]:
print("Number of training examples = ", str(X_train.shape[0]))
print("Number of test examples = ", str(X_test.shape[0]))
print("X_train shape = ", str(X_train.shape))
print("Y_train shape = ", str(Y_train.shape))
print("X_test shape = ", str(X_test.shape))
print("Y_test shape = ", str(Y_test.shape))

Number of training examples =  5000
Number of test examples =  1000
X_train shape =  (5000, 2500)
Y_train shape =  (1, 5000)
X_test shape =  (1000, 2500)
Y_test shape =  (1, 1000)


In [9]:
train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)

train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])

X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)

print("X train shape: ", str(X_train.shape))
print("X test shape: ", str(X_test.shape))
print("Y train shape: ", str(train_y.shape))

X train shape:  (5000, 50, 50, 1)
X test shape:  (1000, 50, 50, 1)
Y train shape:  (5000, 6)


In [10]:
def keras_model(image_x, image_y):
    num_of_classes = 6
    model = Sequential() #EAch step is followed by next step
    #Add convolutional layer with 32 diff filters. Each filter is responsible for getting 1 shape.
    #Relu performs very well. Activation can be sigmoid as well
    model.add(Conv2D(32, (5,5), input_shape=(image_x, image_y, 1), activation='relu'))
    #Next layer is MaxPooling layer. Reduces dimensionality
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    # Conv layer of 64 filters
    model.add(Conv2D(64, (5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    #If they're matrix, then convert to vector
    model.add(Flatten())
    # Hidden input layers is 1024
    model.add(Dense(1024, activation='relu'))
    # Dropout helps in reducing overfitting. I dont want to fit too much to training set
    # Only 0.6% of hidden units will be active, rest will be inactive
    model.add(Dropout(0.6))
    # Softmax when you want to generalise output into n number of classes
    model.add(Dense(num_of_classes, activation='softmax'))
    # crossentropy is used for softmax activation
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath= "handEmo.h5"
    
    #Checkpoint used in training a very big model
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    
    return model, callbacks_list

In [12]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64,
         callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" %(100 - scores[1] * 100))
print_summary(model)

model.save('handEmo.h5')

W0814 17:46:42.309371 22576 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 5000 samples, validate on 1000 samples
Epoch 1/10
5000/5000 [==============================] - 17s 3ms/step - loss: 0.1686 - acc: 0.9448 - val_loss: 3.6178e-04 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to handEmo.h5
Epoch 2/10
5000/5000 [==============================] - 16s 3ms/step - loss: 4.1161e-04 - acc: 1.0000 - val_loss: 3.5768e-05 - val_acc: 1.0000

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/10
5000/5000 [==============================] - 16s 3ms/step - loss: 4.2890e-04 - acc: 0.9998 - val_loss: 2.0551e-05 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/10
5000/5000 [==============================] - 16s 3ms/step - loss: 3.7403e-04 - acc: 1.0000 - val_loss: 2.2730e-05 - val_acc: 1.0000

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/10
5000/5000 [==============================] - 16s 3ms/step - loss: 6.9215e-05 - acc: 1.0000 - val_loss: 3.8697e-06 - val_acc: 1.0000

Epoch 0